### Download Glove Embeddings

In [1]:
import os

GLOVE_PATH = os.path.join("glove", "glove.840B.300d.txt")

if not os.path.exists(GLOVE_PATH):
    !wget "https://nlp.stanford.edu/data/glove.840B.300d.zip"
    !unzip "glove.840B.300d.zip" -d "glove"
    !rm "glove.840B.300d.zip"

### Install dependencies

In [ ]:
!pip install -r requirements.txt
!python -m spacy download en_core_web_sm

### Create the datasets for training 

In [2]:
%load_ext autoreload
%autoreload 2

from train_module.train import train_model
from dataset.snli import SNLIDataset
from dataset.dataset_utils import get_glove_embeddings
from dataset.dataset_utils import get_glove_vocab_index_mapping
from torch.utils.data import DataLoader

glove_vocab_index_mapping = get_glove_vocab_index_mapping(GLOVE_PATH)
glove_embeddings = get_glove_embeddings(GLOVE_PATH)

train_dataset = SNLIDataset("train", glove_vocab_index_mapping)
validation_dataset = SNLIDataset("validation", glove_vocab_index_mapping)
test_dataset = SNLIDataset("test", glove_vocab_index_mapping)

BATCH_SIZE = 64
NUM_WORKERS = 8

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS
)

validation_loader = DataLoader(
    validation_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS
)


/home/andreib/miniconda3/envs/atcs/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset snli (/home/andreib/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)
100%|██████████| 3/3 [00:00<00:00, 47.21it/s]
Found cached dataset snli (/home/andreib/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)
100%|██████████| 3/3 [00:00<00:00, 598.47it/s]
Found cached dataset snli (/home/andreib/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)
100%|██████████| 3/3 [00:00<00:00, 1344.47it/s]


### Train all the model configurations

In [4]:
models_output_size = {
    "BaselineEncoder": {
        "output_size": 1200,
        "bidirectional": False
    },
    "BaselineLSTM": {
        "output_size": 1200,
        "bidirectional": False
    },
    "BidirectionalLSTM": {
        "output_size": 2400,
        "bidirectional": True
    },
    "MaxPoolingBiLSTM": {
        "output_size": 2400,
        "bidirectional": True
    }
}

for model_name, config in models_output_size.items():
    output_size, bidirectional = config.values()

    train_model(
        model_name,
        data_loaders=[train_loader, validation_loader, test_loader],
        glove_embeddings=glove_embeddings,
        num_embeddings=glove_embeddings.shape[0],
        embedding_dim=glove_embeddings.shape[1],
        bidirectional=bidirectional,
        encoder_output_size=output_size
    )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/andreib/miniconda3/envs/atcs/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
Global seed set to 42
